In [73]:
import pandas as pd
import pyodbc
from datetime import datetime
from dateutil.relativedelta import relativedelta
import psycopg2
import useful_functions as use

In [ ]:
#fips is 36103

In [2]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [44]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [46]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [3]:
QUERY = \
    f"""
    SELECT 
    ZIP_CD,
    ZIP_PLUS_4,
    FIPS_CD
    FROM
    AMAEDW.GEODATA_ZR
    WHERE
    FIPS_CD = 36103
    """
zips = pd.read_sql(con=AMAEDW, sql=QUERY)

In [8]:
suffolk_zips = tuple(zips.ZIP_CD.unique())

In [13]:
PURPOSE_QUERY = \
    """
    SELECT *
    FROM 
    AMAEDW.CONT_PURPOSE_TYPE
    """
purpose = pd.read_sql(con=AMAEDW, sql=PURPOSE_QUERY)

In [11]:
PARTY_ADDR_QUERY = \
    f"""
    SELECT 
    P.POST_CD,
    P.POST_CD_PLUS_4,
    A.POST_CD_ID,
    A.PARTY_ID,
    A.FROM_DT,
    A.THRU_DT,
    A.CAT_CD_ID,
    A.PURPOSE_TYPE_ID,
    A.PARTY_ADDR_ID
    FROM
    AMAEDW.PARTY_ADDR A, AMAEDW.POST_CD P
    WHERE
    P.POST_CD_ID = A.POST_CD_ID
    AND
    P.POST_CD in {suffolk_zips}
    """
party_addr = pd.read_sql(con=AMAEDW, sql=PARTY_ADDR_QUERY)

In [16]:
party_addr = pd.merge(purpose[['PURPOSE_TYPE_ID','PURPOSE_USG_DESC','PURPOSE_USG_CD']], party_addr, on='PURPOSE_TYPE_ID')

In [17]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [95]:
QUERY = \
    """
    SELECT
    PARTY_ID,
    TOP_ID
    FROM
    AMAEDW.MED_PROF
    WHERE
    THRU_DT is null
    """
tops = pd.read_sql(con=AMAEDW, sql=QUERY)

In [30]:
suffolk_peeps = pd.merge(party_ids, party_addr, on='PARTY_ID')

In [97]:
PEEPS = pd.merge(party_addr, tops, on='PARTY_ID')

In [31]:
suffolk_peeps = suffolk_peeps[suffolk_peeps.PURPOSE_USG_CD.isin(['PP  ','PO  '])]

In [98]:
PEEPS = PEEPS[PEEPS.PURPOSE_USG_CD.isin(['PP  ','PO  '])]
current_suffolk_2 = PEEPS[PEEPS.THRU_DT.isna()]

In [47]:
current_suffolk = suffolk_peeps[suffolk_peeps.THRU_DT.isna()]

In [38]:
today_date = datetime.today()
ten_year = (today_date - relativedelta(years=10))

In [86]:
old_suffolk = suffolk_peeps[(suffolk_peeps.FROM_DT<ten_year)&(suffolk_peeps.THRU_DT>ten_year)].drop_duplicates('ME')

In [85]:
still_suffolk = suffolk_peeps[(suffolk_peeps.FROM_DT<ten_year)&(suffolk_peeps.THRU_DT.isna())].drop_duplicates('ME')

In [49]:
current_suffolk = pd.merge(current_suffolk, ov_me, left_on='ME', right_on='medical_education_number')

In [120]:
current_suffolk = pd.merge(current_suffolk, TOPS, on='ME')

In [126]:
current_suffolk[(current_suffolk.PURPOSE_USG_CD=='PO  ')&(current_suffolk.TOP_ID==117)].drop_duplicates('ME')

,PARTY_ID_x,ME,PURPOSE_TYPE_ID,PURPOSE_USG_DESC,PURPOSE_USG_CD,POST_CD,POST_CD_PLUS_4,POST_CD_ID,FROM_DT,THRU_DT,CAT_CD_ID,PARTY_ADDR_ID,medical_education_number,type,PARTY_ID_y,TOP_ID
0,1891161,00502870203,317,Preferred Office,PO,11794,3469,30600334,2020-02-05 21:08:57.783313,NaT,3923.0,-1,00502870203,Physician,1891161,117.0
2,1895781,00502761005,317,Preferred Office,PO,11743,2787,28462357,2018-02-08 20:44:54.544269,NaT,4426.0,-1,00502761005,Physician,1895781,117.0
7,1898785,00511771404,317,Preferred Office,PO,11787,3400,29183657,2021-12-10 07:24:48.132509,NaT,4159.0,-1,00511771404,Physician,1898785,117.0
9,1893901,00511930022,317,Preferred Office,PO,11740,1624,30890432,2021-12-10 07:24:48.883814,NaT,4159.0,-1,00511930022,Physician,1893901,117.0
12,1896940,00514801100,317,Preferred Office,PO,11747,2358,21785957,2021-12-10 07:24:52.437264,NaT,4159.0,-1,00514801100,Physician,1896940,117.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12565,12839743,03575170896,317,Preferred Office,PO,11704,8308,33995899,2021-12-10 07:29:23.696230,NaT,6455.0,-1,03575170896,Physician,12839743,117.0
12570,12839770,03575170934,317,Preferred Office,PO,11795,4927,10696032,2018-08-03 20:51:21.991497,NaT,6455.0,-1,03575170934,Physician,12839770,117.0
12574,9882538,42201180214,317,Preferred Office,PO,11776,8055,32814926,2021-11-26 07:16:19.301436,NaT,6455.0,-1,42201180214,Physician,9882538,117.0
12615,13048121,04878151343,317,Preferred Office,PO,11795,4927,10696032,2018-09-21 21:08:26.660637,NaT,6455.0,-1,04878151343,Physician,13048121,117.0


In [55]:
current_suffolk.PURPOSE_USG_CD.unique()

array(['PO  ', 'PP  '], dtype=object)

In [60]:
ye_pcd = pd.read_csv('../../Data/OLD_FILES/YE_PCD_2013.csv')

In [63]:
for col in ye_pcd.columns:
    print(col)

mpa
amamem
menumber
del
cut
dead
newname
stuflg
do
tfgflg
sex
mflag
changeme
ua
pdel
uncut
pd
blan1
blan2
dsegflg
hospriv
pendama
blan3
blan4
blan5
fifthp
osteo
blan6
amastaff
nc
doi
natass
blan7
nr
region
division
group
fed
dopsp
blan8
actup
changead
changeso
gradcent
adjtop
nameo
suffix
labname
address2
blan9
address3
zipcode
geostate
county
city
blan10
birthdat
birthpl
citz
visa
ecfmg
licyr
natbrdyr
update
statemem
top
tophrs
pe
pehrs
spec1
sp1hrs
spec2
sp2hrs
spec3
tranyr
board1
board2
board3
specsoc1
specsoc2
specsoc3
specsoc4
specsoc5
specsoc6
b
curmedtr
curmedsp
curmeds2
tofrom
trinst
govserv
profapt
profapsp
medschool
med_grad
cmemstat
priorsta
pre2stat
blan11
blan12
curyear
blan13
amamemst
blan14
z
pamastyr
q
p2amasty
w
addline2
v
socsecnum
curprod
priorprd
p2pcode
blan15
lastmemy
lastmemp
lmemgeo
lmemstat
blan16
effortcd
blan17
AGE
STATE_CD
STATE_NM


In [77]:
ye_pcd['ZIP_CD'] = [use.fix_zipcode(x) for x in ye_pcd.zipcode]

In [83]:
for_fun = use.fix_me(ye_pcd[ye_pcd.ZIP_CD.isin(suffolk_zips)].menumber)

In [87]:
pd.concat([still_suffolk, old_suffolk])

,PARTY_ID,ME,PURPOSE_TYPE_ID,PURPOSE_USG_DESC,PURPOSE_USG_CD,POST_CD,POST_CD_PLUS_4,POST_CD_ID,FROM_DT,THRU_DT,CAT_CD_ID,PARTY_ADDR_ID
2,1890648,00502450042,269,Preferred Professional Mailing Address,PP,11733,3102,10613482,2006-03-29 23:43:34.233755,NaT,NaN,0
66,1893091,00511410118,317,Preferred Office,PO,11973,None,10171142,2005-12-16 20:31:45.508892,NaT,NaN,0
567,1910746,00502820613,269,Preferred Professional Mailing Address,PP,11776,2212,10336423,2005-12-16 20:23:24.138841,NaT,NaN,0
597,1898776,00511770084,269,Preferred Professional Mailing Address,PP,11777,2203,10444007,2005-12-16 20:22:36.345200,NaT,NaN,0
638,1902760,00515630287,269,Preferred Professional Mailing Address,PP,11780,1228,20430995,2010-03-23 00:05:03.993804,NaT,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
209454,6289003,03575080749,317,Preferred Office,PO,11795,4927,10696032,2012-05-25 04:04:56.857876,2014-02-20 21:24:53,NaN,-1
209475,6298005,04204130707,328,Preferred Professional Mailing Address,PP,11720,2847,22390282,2012-05-10 23:48:42.422016,2012-11-10 05:37:04,NaN,-1
209479,4624200,30501096128,309,Preferred Professional Mailing Address,PP,11706,8408,10597536,2012-05-12 08:25:50.346508,2012-07-17 23:37:59,NaN,-1
209496,6314001,03575052273,318,Preferred Professional Mailing Address,PP,11702,3028,21778329,2012-06-05 01:51:57.502906,2014-07-15 00:58:49,NaN,-1


In [94]:
ye_pcd[ye_pcd.ZIP_CD.isin(suffolk_zips)&(ye_pcd.top==12)]

,mpa,amamem,menumber,del,cut,dead,newname,stuflg,do,tfgflg,...,lastmemp,lmemgeo,lmemstat,blan16,effortcd,blan17,AGE,STATE_CD,STATE_NM,ZIP_CD
660984,HPR,PM,3.508132e+09,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,26,NY,New York,11786
661019,HPR,PM,3.546120e+09,0,0,0,0,0,0.0,0.0,...,70,35.0,ACTIVE,0,NaN,NaN,28,NY,New York,11767
661082,HPR,NM,3.509121e+09,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,27,NY,New York,11794
661149,HPR,PM,4.971310e+10,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,30,NY,New York,11790
661181,HPR,NM,3.050110e+10,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,30,NY,New York,11706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756000,HPR,PM,4.113121e+09,0,0,0,0,0,0.0,0.0,...,07,80.0,ACTIVE,0,NaN,NaN,28,NY,New York,11794
756004,HPR,NM,4.220114e+10,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,27,NY,New York,11794
756025,HPR,NM,5.500213e+10,0,0,0,0,0,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,27,NY,New York,11755
756075,HPR,PM,3.506131e+09,0,0,0,0,0,0.0,0.0,...,70,35.0,ACTIVE,0,NaN,NaN,26,NY,New York,11777


In [113]:
TOPS = pd.merge(tops, party_ids, on='PARTY_ID')

In [139]:
current_suffolk_2 = pd.merge(current_suffolk_2, party_ids, on='PARTY_ID')

In [140]:
current_suffolk_2[~current_suffolk_2.ME.isin(ov_me.medical_education_number)]

,PURPOSE_TYPE_ID,PURPOSE_USG_DESC,PURPOSE_USG_CD,POST_CD,POST_CD_PLUS_4,POST_CD_ID,PARTY_ID,FROM_DT,THRU_DT,CAT_CD_ID,PARTY_ADDR_ID,TOP_ID,ME
82,317,Preferred Office,PO,11733,4079,10669408,2147917,2005-12-16 20:54:05.040201,NaT,NaN,0,113.0,02301620295
83,318,Preferred Professional Mailing Address,PP,11733,4079,10669408,2147917,2005-12-16 20:53:21.323673,NaT,NaN,0,113.0,02301620295
97,269,Preferred Professional Mailing Address,PP,11706,8759,23223339,2187069,2013-06-17 21:24:26.715443,NaT,NaN,-1,113.0,03503530227
150,269,Preferred Professional Mailing Address,PP,11795,4619,10369223,2346129,2005-12-16 21:12:36.510402,NaT,NaN,0,113.0,03508430654
151,317,Preferred Office,PO,11795,4619,10074186,2346129,2005-12-16 21:10:00.607965,NaT,NaN,0,113.0,03508430654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13975,318,Preferred Professional Mailing Address,PP,11743,0559,10387928,2363659,2005-12-16 21:12:45.758078,NaT,NaN,0,113.0,03546620041
13976,318,Preferred Professional Mailing Address,PP,11794,0001,10432630,2824121,2005-12-16 22:06:16.515276,NaT,NaN,0,117.0,70402870123
13977,318,Preferred Professional Mailing Address,PP,11768,2200,10510988,2725294,2005-12-16 22:05:44.374249,NaT,NaN,0,117.0,33003650036
13979,318,Preferred Professional Mailing Address,PP,11754,None,10555463,2710324,2005-12-16 22:06:52.247764,NaT,NaN,0,114.0,27501600038


In [127]:
PPD = use.get_ppd()

INFO:useful_functions:PPD file from Mar 05 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [145]:
ye_pcd[ye_pcd.menumber==3503530227].AGE

706786    86
Name: AGE, dtype: int64

In [135]:
PPD[PPD.ME=='00102000218']

AttributeError: 'DataFrame' object has no attribute 'labname'